In [1]:
import os
import glob
from collections import OrderedDict

import matplotlib as mpl    
import matplotlib.pyplot as plt    
import matplotlib.gridspec
mpl.rcParams['figure.figsize'] = (10.0, 6.0)
mpl.rcParams['font.size'] = 14
mpl.rcParams['savefig.dpi'] = 72

import numpy as np

from astropy.io import fits
import astropy.wcs 
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
    
import drizzlepac

import pysynphot as S

import grizli
import grizli.fake_image

print('\n grizli version: %s' %(grizli.__version__))

%matplotlib inline


The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       imagefindpars           mapreg              photeq       
     pixreplace           pixtopix            pixtosky        refimagefindpars  
     resetbits          runastrodriz          skytopix           tweakback      
      tweakreg           updatenpol

 grizli version: 1.0.dev1398


In [2]:
# Work in /tmp for now
os.chdir('/tmp/')

In [12]:

## Fetch WFC3/IR F140W images of the UDF
if not os.path.exists('hlsp_xdf_hst_wfc3ir-60mas_hudf_f140w_v1_sci.fits'):
    url = 'https://archive.stsci.edu/missions/hlsp/xdf/'
    wget='wget --no-check-certificate'
    os.system('%s %s/hlsp_xdf_hst_wfc3ir-60mas_hudf_f140w_v1_sci.fits' %(wget, url))
    os.system('%s %s/hlsp_xdf_hst_wfc3ir-60mas_hudf_f140w_v1_wht.fits' %(wget, url))
    
## 3D-HST photometry around the HUDF field, and an extension with best-fit photo-z SED
if not os.path.exists('udf_3dhst_cat.fits'):
    os.system('wget http://www.stsci.edu/~brammer/Grizli/Demos/udf_3dhst_cat.fits')

In [14]:
## Make an object catalog / segmentation image with photutils
sci = fits.open('hlsp_xdf_hst_wfc3ir-60mas_hudf_f140w_v1_sci.fits')
wht = fits.open('hlsp_xdf_hst_wfc3ir-60mas_hudf_f140w_v1_wht.fits')
rms = 1/np.sqrt(wht[0].data)
rms[~np.isfinite(rms)] = 1000
dq = rms > 900
wcs = astropy.wcs.WCS(sci[0].header)

# Run the detection with the grizli / photutils wrapper
from grizli.utils import detect_with_photutils
cat, seg = detect_with_photutils(sci[0].data, err=rms, dq=dq, seg=None, 
                                 detect_thresh=1.4, npixels=6, grow_seg=3, 
                                 gauss_fwhm=2.0, gsize=3, wcs=wcs, save_detection=False, 
                                 root='udf_f140w_photutils', background=None, gain=None, 
                                 AB_zeropoint=26.452, 
                                 rename_columns={'ycentroid': 'y_flt', 'xcentroid': 'x_flt', 
                                                 'dec_icrs_centroid': 'dec', 
                                                 'ra_icrs_centroid': 'ra'}, 
                                 verbose=True)

# code expects this columns later....
cat['NUMBER'] = cat['id']

udf_f140w_photutils: photutils.detect_sources (detect_thresh=1.4, grow_seg=3, gauss_fwhm=2.0, ZP=26.5)
udf_f140w_photutils: photutils.source_properties
Rename column: xcentroid -> x_flt
Rename column: ycentroid -> y_flt
udf_f140w_photutils: photutils.source_properties - 3354 objects


In [20]:
# Get matches from 3D-HST catalog
ref_3dhst = fits.open('udf_3dhst_cat.fits')
ref_cat = Table.read(ref_3dhst[1])

gs = SkyCoord(ra=ref_cat['ra']*u.degree, dec=ref_cat['dec']*u.degree)
cat_rd = SkyCoord(ra=cat['sky_centroid'].ra, dec=cat['sky_centroid'].dec)

gs_idx, d2d, d3d = cat_rd.match_to_catalog_sky(gs)
has_gs_match = np.where(d2d < 2*u.arcsec)[0]

# Use 3D-HST mags because quick photutils catalog has some issues, 
# perhaps related to background subtraction
gs_mag = 25-2.5*np.log10(ref_cat['F204'])
cat['MAG_AUTO'] = gs_mag[gs_idx] 
cat.write('udf_f140w_photutils.cat', format='ascii.commented_header')

fits.writeto('udf_f140w_photutils_seg.fits', data=np.cast[int](seg), 
             header=sci[0].header, clobber=True)

In [30]:
## Setup fake images, cendered in the UDF/XDF
ra, dec = 53.1592277508136, -27.782056346146
pa_aper = 128.589

np.random.seed(1)
# Crude exposure parameters.  Rough read noise and backgrounds coded in 
# grizli.fake_image to make some reasonable noise estimate
EXPTIME = 1.e4 # 10 ks ~ 4 HST orbits
NEXP = 10      # divided between 10 exposures

# JWST NIRISS, three filters & two orients
for filt in ['F356W', 'F444W', 'F410M']:
    for theta in [0,90]:
        h, wcs = grizli.fake_image.nircam_header(filter=filt, ra=ra, dec=dec, 
                                                 pa_aper=pa_aper+theta)
        print('Filter: {filter}, Background: {bg} e/s/pix, RN: {RN} e/exp'.format(filter=filt, 
                                                        bg=h['BACKGR'], RN=h['READN']))
        output = 'nircam_{filt}_{theta:02d}_flt.fits'.format(filt=filt, theta=theta)
        grizli.fake_image.make_fake_image(h, output=output, exptime=EXPTIME, nexp=NEXP)

Filter: F356W, Background: 0.9 e/s/pix, RN: 9 e/exp
Filter: F356W, Background: 0.9 e/s/pix, RN: 9 e/exp
Filter: F444W, Background: 3.0 e/s/pix, RN: 9 e/exp
Filter: F444W, Background: 3.0 e/s/pix, RN: 9 e/exp
Filter: F410M, Background: 0.5 e/s/pix, RN: 9 e/exp
Filter: F410M, Background: 0.5 e/s/pix, RN: 9 e/exp


In [3]:
glob.glob('n*')

['nircam_F356W_00_flt.fits',
 'nircam_F356W_90_flt.fits',
 'nircam_F410M_00_flt.fits',
 'nircam_F410M_90_flt.fits',
 'nircam_F444W_00_flt.fits',
 'nircam_F444W_90_flt.fits',
 'niriss_F356W_00_flt.fits',
 'niriss_F356W_90_flt.fits',
 'niriss_F410M_00_flt.fits',
 'niriss_F410M_90_flt.fits',
 'niriss_F444W_00_flt.fits',
 'niriss_F444W_90_flt.fits']

In [4]:
pwd

'/private/tmp'

In [6]:
from grizli import multifit
# Load GroupFLT for simulation, NB: input files are just noise
sim = multifit.GroupFLT(grism_files=glob.glob('nircam_*flt.fits'), direct_files=[], 
                               ref_file='hlsp_xdf_hst_wfc3ir-60mas_hudf_f140w_v1_sci.fits', 
                               seg_file='udf_f140w_photutils_seg.fits', 
                               catalog='udf_f140w_photutils.cat',
                                pad=200)

UnboundLocalError: local variable 'conf_file' referenced before assignment

In [ ]:
sim.compute_full_model(mag_limit=27)


In [ ]:
# Compute model grism spectra for 3D-HST matches based on full photo-z templates
detection_bp = S.ObsBandpass('wfc3,ir,f140w')
for ix in has_gs_match:
    templ = ref_3dhst['WAVE'].data*(1+ref_cat['zbest'][gs_idx[ix]])
    tempf = ref_3dhst['FLAMBDA'].data[gs_idx[ix],:]
    # Needs to be normalized to unity in the detection band
    spec = S.ArraySpectrum(wave=templ, flux=tempf, waveunits='angstroms', fluxunits='flam')
    spec = spec.renorm(1., 'flam', detection_bp)

    id = cat['id'][ix]
    #print(id)
    sim.compute_single_model(id, mag=cat['MAG_AUTO'][ix], size=-1, store=False, 
                             spectrum_1d=[spec.wave, spec.flux], get_beams=None, 
                             in_place=True)

In [ ]:

# Blotted reference image 
# "grism" exposures are still empty, just noise
fig = plt.figure(figsize=[9,9*2./3])
for ix, i in enumerate([0,2,4,1,3,5]):
    ax = fig.add_subplot(2,3,ix+1)
    ax.imshow(sim.FLTs[i].direct['REF'], vmin=-0.01, vmax=0.05, cmap='viridis', 
              origin='lower')
    ax.set_xticklabels([]); ax.set_yticklabels([])
    ax.grid(color='w', alpha=0.8)
    ax.text(100,100,sim.FLTs[i].grism_file, color='w', size=10, ha='left', va='bottom')
fig.tight_layout(pad=0.1)

In [27]:
sim.FLTs

[]